script to run full preprocessing pipeline on one patientID.

In [9]:
import pickle
import pandas as pd
import numpy as np
import nrrd

import matplotlib.pyplot as plt

import subprocess
import os
import glob
import shutil

import time
import datetime

print(datetime.datetime.now())

2021-12-27 11:42:10.176234


In [2]:
path_data = '../data/d20211203/training/'

# INPUTS
path_masks = path_data + '0raw_masks/'
path_dcmzips = path_data + '0raw_dicom/'
path_pngs = path_data + '0raw_png/'
path_meta = path_data + 'nrrds_metadata/'
path_nrrds = path_data + 'nrrds_matched2mask/'

# TEMP PATHS
temp_unzip = path_data + 'temp_unzipSingle/'
temp_target = path_data + 'temp_outputs/'

# OUTPUTS


In [3]:
# prepare directories
try:
    os.mkdir(temp_unzip)
except OSError:
    print ("Creation of the directory %s failed" % temp_unzip)
else:
    print ("Successfully created the directory %s " % temp_unzip)
    
try:
    os.mkdir(temp_target)
except OSError:
    print ("Creation of the directory %s failed" % temp_target)
else:
    print ("Successfully created the directory %s " % temp_target)

Successfully created the directory data/d20211203/training/temp_unzipSingle/ 
Creation of the directory data/d20211203/training/temp_outputs/ failed


In [12]:
# define paths
patientID = '1094'
fn_dcmzip = path_dcmzips + str(patientID) + '.zip'
fn_meta = path_meta + str(patientID) + '.nrrd'
fn_nrrd = path_nrrds + str(patientID) + '.nrrd'
dir_png = path_pngs + str(patientID)

# define output paths
path_metaTarget = temp_target + str(patientID) + '_meta.nrrd'
path_nrrdTarget = temp_target + str(patientID) + '.nrrd'

## step 0

In [14]:
# unzip
cmd = 'unzip %s -d %s' %(fn_dcmzip, temp_unzip)   # unzip WHAT -d WHERE
subprocess.run(cmd, shell=True)

CompletedProcess(args='unzip data/d20211203/training/0raw_dicom/1094.zip -d data/d20211203/training/temp_unzipSingle/', returncode=0)

In [16]:
# grab meta data
# identify signs of hidden folders
garbageIndicators = ["__MACOSX", "/."]

# use os.walk to get names of all directories
listDir = []
for root, dirs, files in os.walk(temp_unzip, topdown=False):
    for name in [x for x in dirs]:
        listDir.append(os.path.join(root, name))
# remove directories that are hidden folders
notHiddenDir = [s for s in listDir if not any(x in s for x in garbageIndicators)]
# check if dicom files are in deepest directory
potentialDir = max(notHiddenDir)
potentialFiles = [s for s in os.listdir(potentialDir) if '.dcm' in s and not s.startswith('.')]
if potentialFiles: # if potentialFiles is not empty
    path_dcm = potentialDir
    # convert dicom to nrrd
    cmd = "plastimatch convert --input %s --output-img %s"%(path_dcm, path_metaTarget)
    output = subprocess.run(cmd, shell=True, capture_output=True)
output

CompletedProcess(args='plastimatch convert --input data/d20211203/training/temp_unzipSingle/000732__F_1900-01-01_000732_DEFAULT/1.2.826.0.1.3680043.8.760.0.2041749483.1627387376225.11772/1.2.826.0.1.3680043.8.760.0.2041749483.1627387376225.11773 --output-img data/d20211203/training/temp_outputs/1094_meta.nrrd', returncode=0, stdout=b'Setting PIH from RDD\nPIH is:\nOrigin = -177.6777 -356.6777 -629.5000\nSize = 512 512 242\nSpacing = 0.6445 0.6445 2.0000\nDirection = 1.0000 0.0000 0.0000 0.0000 1.0000 0.0000 0.0000 0.0000 1.0000\nRt_study_warp: Saving m_img (data/d20211203/training/temp_outputs/1094_meta.nrrd)\nTrying to write image to data/d20211203/training/temp_outputs/1094_meta.nrrd\nRt_study_warp: warp and save ss.\nFinished!\n', stderr=b'')

## step 1

In [17]:
# read metadata
meta_nrrd = nrrd.read(path_metaTarget)

In [21]:
# convert png to numpy array
fns_pngs = np.array([x for x in os.listdir(dir_png) if x.endswith(".png") and not x.startswith("._")])
i_pngs = np.array([x.replace(".png", "") for x in fns_pngs], dtype = int)
image = np.array([plt.imread(os.path.join(dir_png, x)) for x in fns_pngs[np.argsort(i_pngs)]])
image = image.transpose()

currMeta = meta_nrrd[1]
currMeta["type"] = "int"

nrrd.write(path_nrrdTarget, image, header=currMeta)

KeyError: 'O'

In [22]:
image

array([array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
       array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
       array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
       array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],

## step 2

In [ ]:
# convertpng2nrrd


In [ ]:
# cleanup
shutil.rmtree(temp_unzip)
shutil.rmtree(temp_target)